# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [73]:
# YOUR CHANGES HERE
import cvxpy as cp
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 

import yfinance as yf 

tickers = ['AAPL', 'AMZN', 'SPY', 'TSLA','IBM','JNJ']

historical_prices = None

for t in tickers:
    ticker = yf.Ticker(t)
    history = ticker.history(period='2y')

    monthly_prices = history['Close'].resample('ME').last()

    if historical_prices is None:
        historical_prices = monthly_prices
    else:
        historical_prices = pd.concat([historical_prices, monthly_prices], axis = 1)
...

historical_prices.columns = tickers
historical_prices.index = pd.to_datetime(historical_prices.index).strftime('%Y-%m')

historical_prices

,AAPL,AMZN,SPY,TSLA,IBM,JNJ
Date,,,,,,
2023-09,169.549271,127.120003,417.865662,250.220001,131.234116,146.245300
2023-10,169.113556,133.089996,408.794342,200.839996,135.293686,139.287506
2023-11,188.355331,146.089996,446.135223,240.080002,149.996246,146.384766
2023-12,190.913651,151.940002,466.503662,248.479996,154.716751,148.353470
2024-01,182.851898,155.199997,473.933441,187.289993,173.740601,150.397873
2024-02,179.461258,176.759995,498.666504,201.880005,176.632401,153.904953
2024-03,170.257339,180.380005,514.973877,175.789993,182.293274,150.862717
2024-04,169.115570,175.000000,494.210327,183.279999,158.656982,137.892654
2024-05,191.138153,176.440002,519.207397,178.080002,160.858627,141.006973


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [74]:
# YOUR CHANGES HERE
historical_prices.index.names = ['date']
historical_prices.to_csv('historical_prices.tsv', sep='\t')


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [75]:
# YOUR CHANGES HERE

historical_returns = historical_prices.pct_change().dropna()
historical_returns

,AAPL,AMZN,SPY,TSLA,IBM,JNJ
date,,,,,,
2023-10,-0.002570,0.046963,-0.021709,-0.197346,0.030934,-0.047576
2023-11,0.113780,0.097678,0.091344,0.195379,0.108671,0.050954
2023-12,0.013582,0.040044,0.045655,0.034988,0.031471,0.013449
2024-01,-0.042227,0.021456,0.015927,-0.246257,0.122959,0.013781
2024-02,-0.018543,0.138918,0.052187,0.077901,0.016644,0.023319
2024-03,-0.051286,0.020480,0.032702,-0.129235,0.032049,-0.019767
2024-04,-0.006706,-0.029826,-0.040320,0.042608,-0.129661,-0.085973
2024-05,0.130222,0.008229,0.050580,-0.028372,0.013877,0.022585
2024-06,0.095553,0.095273,0.035280,0.111186,0.036560,-0.003477


Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [76]:
# YOUR CHANGES HERE
historical_returns.to_csv('historical_returns.tsv', sep='\t')


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [77]:
# YOUR CHANGES HERE

estimated_returns = historical_returns.mean().to_frame('estimated_return')
estimated_returns.index.name = 'asset'
estimated_returns

,estimated_return
asset,
AAPL,0.016215
AMZN,0.027565
SPY,0.018960
TSLA,0.026225
IBM,0.029902
JNJ,0.009314


Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [78]:
# YOUR CHANGES HERE
estimated_returns.to_csv('estimated_returns.tsv',sep='\t')


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [79]:
# YOUR CHANGES HERE
estimated_covariance = historical_returns.cov()

estimated_covariance.index.name = None
estimated_covariance.columns.name = None
estimated_covariance



,AAPL,AMZN,SPY,TSLA,IBM,JNJ
AAPL,0.003483,0.000420,0.000783,0.003130,0.000103,0.000832
AMZN,0.000420,0.004322,0.001602,0.005318,0.001811,-0.000510
SPY,0.000783,0.001602,0.001248,0.002300,0.001499,0.000558
TSLA,0.003130,0.005318,0.002300,0.025562,0.002026,-0.001298
IBM,0.000103,0.001811,0.001499,0.002026,0.006227,0.000664
JNJ,0.000832,-0.000510,0.000558,-0.001298,0.000664,0.002420


Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [80]:
# YOUR CHANGES HERE
estimated_covariance.to_csv('estimated_covariance.tsv', sep='\t')


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [81]:
# YOUR CHANGES HERE
max_return_asset = estimated_returns['estimated_return'].idxmax()

# Create allocation vector (all zeros)
allocation = pd.Series(0.0, index=estimated_returns.index)
# Put 100% in the max return asset
allocation[max_return_asset] = 1.0

# Convert to DataFrame with proper column names
maximum_return_portfolio = allocation.to_frame('allocation')
maximum_return_portfolio.index.name = 'asset'
maximum_return_portfolio

,allocation
asset,
AAPL,0.0
AMZN,0.0
SPY,0.0
TSLA,0.0
IBM,1.0
JNJ,0.0


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [82]:
# YOUR CHANGES HERE

maximum_return_portfolio.to_csv('maximum_return.tsv', sep='\t')

Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [83]:
# YOUR CHANGES HERE

n = len(estimated_returns)

x_minimum_risk = cp.Variable(n)

objective_minimum_risk = cp.Minimize(x_minimum_risk.T @ estimated_covariance.to_numpy() @ x_minimum_risk)
objective_minimum_risk

prob_minimum_risk = cp.Problem(objective_minimum_risk,
                               [0 <= x_minimum_risk,
                                cp.sum(x_minimum_risk) == 1])

covariance_minimum_risk = prob_minimum_risk.solve()
covariance_minimum_risk

estimated_return_minimum_risk = x_minimum_risk.value.T @ estimated_returns
estimated_return_minimum_risk

estimated_return    0.016478
dtype: float64

Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [84]:
# YOUR CHANGES HERE
optimal_weights = x_minimum_risk.value

# Create the DataFrame with the optimal weights
minimum_risk_portfolio = pd.DataFrame({
    'allocation': optimal_weights
}, index=estimated_returns.index)

# Set the index name to 'asset'
minimum_risk_portfolio.index.name = 'asset'

# Save to TSV file
minimum_risk_portfolio.to_csv('minimum_risk.tsv', sep='\t')


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [85]:
# YOUR CHANGES HERE
ef_variances = []
ef_returns = []
ef_portfolios = []

min_risk_return = minimum_risk_portfolio.T @ estimated_returns['estimated_return'] 
max_return_return = maximum_return_portfolio.T @ estimated_returns['estimated_return']

for r in np.linspace(min_risk_return, max_return_return, 101):
    x_r = cp.Variable(n)
    
    prob_r = cp.Problem(cp.Minimize(x_r.T @ estimated_covariance.to_numpy() @ x_r),
                       [x_r >= 0,
                        cp.sum(x_r) == 1,
                        x_r.T @ estimated_returns['estimated_return'].values == r])
    
    prob_r.solve()
    
    if x_r.value is not None: 
        ef_variances.append(prob_r.value)
        ef_returns.append(r)
        ef_portfolios.append(x_r.value)


# Convert to numpy array
ef_portfolios = np.asarray(ef_portfolios)
ef_portfolios

array([[ 9.37067704e-02,  3.76490762e-02,  6.04392414e-01,
        -7.69232502e-20, -1.44563132e-18,  2.64251739e-01],
       [ 9.44327118e-02,  4.41612441e-02,  6.05467894e-01,
        -7.72018414e-20, -1.44896818e-18,  2.55938150e-01],
       [ 9.51586532e-02,  5.06734119e-02,  6.06543375e-01,
        -7.74883377e-20, -1.45240149e-18,  2.47624560e-01],
       [ 9.58845946e-02,  5.71855798e-02,  6.07618855e-01,
        -7.77692967e-20, -1.45578889e-18,  2.39310970e-01],
       [ 9.70086395e-02,  6.30974516e-02,  6.06238221e-01,
        -5.77794905e-19,  1.54952069e-03,  2.32106167e-01],
       [ 9.89903321e-02,  6.77160855e-02,  5.99566298e-01,
        -5.69118578e-19,  6.43722517e-03,  2.27290059e-01],
       [ 1.00972025e-01,  7.23347195e-02,  5.92894375e-01,
        -5.60402766e-19,  1.13249297e-02,  2.22473951e-01],
       [ 1.02953717e-01,  7.69533534e-02,  5.86222452e-01,
        -5.51674895e-19,  1.62126341e-02,  2.17657843e-01],
       [ 1.04935410e-01,  8.15719873e-02,  5.795

Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [86]:
# YOUR CHANGES HERE
ef_std_devs = [np.sqrt(var) for var in ef_variances]

# Create the DataFrame
efficient_frontier_df = pd.DataFrame({
    'index': range(len(ef_returns)),
    'return': ef_returns,
    'risk': ef_std_devs
})

# Add asset allocation columns using your existing ef_portfolios
asset_names = estimated_returns.index.tolist()
for i, asset in enumerate(asset_names):
    efficient_frontier_df[asset] = ef_portfolios[:, i]

# Save to TSV file
efficient_frontier_df.to_csv('efficient_frontier.tsv', sep='\t', index=False)



In [87]:
print(efficient_frontier_df.head())


   index                  return      risk      AAPL      AMZN       SPY  \
0      0   [0.01647775313158121]  0.032178  0.093707  0.037649  0.604392   
1      1   [0.01661199109620335]  0.032184  0.094433  0.044161  0.605468   
2      2  [0.016746229060825492]  0.032204  0.095159  0.050673  0.606543   
3      3  [0.016880467025447633]  0.032236  0.095885  0.057186  0.607619   
4      4  [0.017014704990069775]  0.032281  0.097009  0.063097  0.606238   

           TSLA           IBM       JNJ  
0 -7.692325e-20 -1.445631e-18  0.264252  
1 -7.720184e-20 -1.448968e-18  0.255938  
2 -7.748834e-20 -1.452401e-18  0.247625  
3 -7.776930e-20 -1.455789e-18  0.239311  
4 -5.777949e-19  1.549521e-03  0.232106  


Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [89]:
# YOUR CHANGES HERE

n_simulations = 1000
n_samples = 23
max_return_assets = []

mean_returns = estimated_returns['estimated_return'].values
asset_names = estimated_returns.index.tolist()

for i in range(n_simulations):
    random_returns = np.random.multivariate_normal(mean_returns, estimated_covariance, n_samples)

    resampled_mean_returns = np.mean(random_returns, axis=0)

    best_asset_index = np.argmax(resampled_mean_returns)
    best_asset_name = asset_names[best_asset_index]
    max_return_assets.append(best_asset_name)

asset_probabilities = {}
for asset in asset_names:
    count = max_return_assets.count(asset)  
    probability = count / n_simulations
    asset_probabilities[asset] = probability

max_return_prob_df = pd.DataFrame({
    'asset': list(asset_probabilities.keys()),
    'probability': list(asset_probabilities.values())
})




Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [90]:
# YOUR CHANGES HERE

max_return_prob_df.to_csv('max_return_probabilities.tsv', sep='\t', index=False)
max_return_prob_df

,asset,probability
0,AAPL,0.062
1,AMZN,0.190
2,SPY,0.005
3,TSLA,0.368
4,IBM,0.341
5,JNJ,0.034


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.